In [ ]:
 # importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

In [ ]:
!pip install PyDrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1frVuOBo8-C-K4tH1MiPubZIJk0S284MD'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_labels.csv')

In [ ]:
id='17ZqNOr1xUxh7v9WKXuzoVcv4L78A5Na8'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_values.csv')

In [ ]:
id='1drBdiqfCoV24qDmw4hMNiTmxlHGW552e'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test_values.csv')

In [ ]:
train_labels = pd.read_csv('train_labels.csv')

In [ ]:
test_values = pd.read_csv('test_values.csv', dtype= {'land_surface_condition':'category','foundation_type':'category','roof_type':'category','ground_floor_type':'category' \
, 'other_floor_type':'category', 'position':'category','plan_configuration':'category','damage_grade':'category','legal_ownership_status':'category','has_secondary_use':'int8'\
,'has_superstructure_adobe_mud': 'int8'\
, 'has_superstructure_mud_mortar_stone': 'int8','has_superstructure_stone_flag': 'int8','has_superstructure_cement_mortar_stone': 'int8','has_superstructure_mud_mortar_brick': 'int8'\
, 'has_superstructure_cement_mortar_brick': 'int8', 'has_superstructure_timber': 'int8', 'has_superstructure_bamboo': 'int8', 'has_superstructure_rc_non_engineered': 'int8', 'has_superstructure_rc_engineered': 'int8'\
, 'has_superstructure_other': 'int8', 'has_secondary_use': 'int8', 'has_secondary_use_agriculture': 'int8', 'has_secondary_use_hotel': 'int8', 'has_secondary_use_rental': 'int8', 'has_secondary_use_institution': 'int8'\
, 'has_secondary_use_school': 'int8', 'has_secondary_use_industry': 'int8', 'has_secondary_use_health_post': 'int8', 'has_secondary_use_gov_office': 'int8', 'has_secondary_use_use_police': 'int8', 'has_secondary_use_other': 'int8'})

In [ ]:
train_values = pd.read_csv('train_values.csv', dtype= {'land_surface_condition':'category','foundation_type':'category','roof_type':'category','ground_floor_type':'category' \
, 'other_floor_type':'category', 'position':'category','plan_configuration':'category','damage_grade':'category','legal_ownership_status':'category','has_secondary_use':'int8'\
,'has_superstructure_adobe_mud': 'int8'\
, 'has_superstructure_mud_mortar_stone': 'int8','has_superstructure_stone_flag': 'int8','has_superstructure_cement_mortar_stone': 'int8','has_superstructure_mud_mortar_brick': 'int8'\
, 'has_superstructure_cement_mortar_brick': 'int8', 'has_superstructure_timber': 'int8', 'has_superstructure_bamboo': 'int8', 'has_superstructure_rc_non_engineered': 'int8', 'has_superstructure_rc_engineered': 'int8'\
, 'has_superstructure_other': 'int8', 'has_secondary_use': 'int8', 'has_secondary_use_agriculture': 'int8', 'has_secondary_use_hotel': 'int8', 'has_secondary_use_rental': 'int8', 'has_secondary_use_institution': 'int8'\
, 'has_secondary_use_school': 'int8', 'has_secondary_use_industry': 'int8', 'has_secondary_use_health_post': 'int8', 'has_secondary_use_gov_office': 'int8', 'has_secondary_use_use_police': 'int8', 'has_secondary_use_other': 'int8'})


In [ ]:
train_values.set_index("building_id", inplace=True)
train_labels.set_index("building_id", inplace=True)
test_values.set_index("building_id", inplace=True)

#encodeo de datos categoricos

In [ ]:
!pip install category_encoders

     |████████████████████████████████| 80 kB 3.0 MB/s 


In [ ]:
import category_encoders as ce
categoricals = ["land_surface_condition", "foundation_type", "roof_type", "ground_floor_type", "other_floor_type", "position", "plan_configuration", "legal_ownership_status" ]
#legal ownership status la sacamos, no?

##Binary Encoding

In [ ]:
encoder = ce.BinaryEncoder(cols=categoricals, return_df=True)
categoricals_encoded = encoder.fit_transform(train_values[categoricals])

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [ ]:
categoricals_encoded.head()

,land_surface_condition_0,land_surface_condition_1,land_surface_condition_2,foundation_type_0,foundation_type_1,foundation_type_2,foundation_type_3,roof_type_0,roof_type_1,roof_type_2,ground_floor_type_0,ground_floor_type_1,ground_floor_type_2,ground_floor_type_3,other_floor_type_0,other_floor_type_1,other_floor_type_2,position_0,position_1,position_2,plan_configuration_0,plan_configuration_1,plan_configuration_2,plan_configuration_3,plan_configuration_4,legal_ownership_status_0,legal_ownership_status_1,legal_ownership_status_2
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
802906,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1
28830,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1
94947,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,1
590882,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,1
201944,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,1


los features estilo `has_secondary_use...` y `has_superstructure...` podrian considerarse como todos un mismo fieature estilo categórico que ya fue encodeado como *One hot Encoding*.  
Podriamos eso encodearlo como Binary encoding y seguramente usar menos columnas/dimensiones.

...Salvo que un mismo edificio puede tener mas de una al mismo tiempo. Habria que ver si igual se puede hacer teniendo alguna salvedad.



En una primer pasada, podemos obviar las distintas posibilidades de secondary use o superstructure y tomarlos todos como una misma cosa  
(o sea, si tiene superstructure o no; si tiene secondary use o no.)

In [ ]:
for col in categoricals:
  del train_values[col]

In [ ]:
encoder = ce.BinaryEncoder(cols=categoricals, return_df=True)
categoricals_encoded_test = encoder.fit_transform(test_values[categoricals])

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [ ]:
for col in categoricals:
  del test_values[col]

In [ ]:
df = train_values.merge(categoricals_encoded, right_index=True, left_index=True)
df.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,land_surface_condition_0,land_surface_condition_1,land_surface_condition_2,foundation_type_0,foundation_type_1,foundation_type_2,foundation_type_3,roof_type_0,roof_type_1,roof_type_2,ground_floor_type_0,ground_floor_type_1,ground_floor_type_2,ground_floor_type_3,other_floor_type_0,other_floor_type_1,other_floor_type_2,position_0,position_1,position_2,plan_configuration_0,plan_configuration_1,plan_configuration_2,plan_configuration_3,plan_configuration_4,legal_ownership_status_0,legal_ownership_status_1,legal_ownership_status_2
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1
28830,8,900,2812,2,10,8,7,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1
94947,21,363,8973,2,10,5,5,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,1
590882,22,418,10694,2,10,6,5,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,1
201944,11,131,1488,3,30,8,9,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,1


In [ ]:
# for preprocessing the data
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KNeighborsClassifier

# for combining the preprocess with model training
from sklearn.pipeline import make_pipeline

# for optimizing the hyperparameters of the pipeline
from sklearn.model_selection import GridSearchCV

In [ ]:
pipe = make_pipeline(StandardScaler(), 
                     KNeighborsClassifier())

In [ ]:
param_grid = {'kneighborsclassifier__n_neighbors': np.arange(1,30),
              'kneighborsclassifier__metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski']}
gs = GridSearchCV(pipe, param_grid, cv=5)

In [ ]:
gs.fit(df, train_labels.values.ravel())


In [ ]:
gs.best_params_


NameError: ignored

In [ ]:
from sklearn.metrics import f1_score

in_sample_preds = gs.predict(df)
f1_score(train_labels, in_sample_preds, average='micro')

In [ ]:
df_test_values = test_values.merge(categoricals_encoded_test, right_index=True, left_index=True)

In [ ]:
predictions = gs.predict(df_test_values)

In [ ]:
submission_format = pd.read_csv('submission_format.csv', index_col='building_id')

In [ ]:
my_submission = pd.DataFrame(data=predictions,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [ ]:
my_submission.to_csv('submission.csv')